## **Laboratorio 4 - Data Science**
### *Mejorando el Análisis de Sentimientos con LSTM y Características Adicionales*
Stefano Aragoni, Carol Arévalo

----------

#### `Importación de Datos`
- Utilice el conjunto de datos IMDB proporcionado por Keras. pero esta vez, en lugar de utilizar sólo las 20.000 palabras más frecuentes,utilice las 50.000 palabras más frecuentes


Como primer paso, se importan las diferentes librerías a utilizar. Principalmente se queriere de Keras, ya que el conjunto de datos IMBD se encuentra en esta librería. Además, se importan las librerías de numpy y pandas para el manejo de los datos.

In [21]:
import tensorflow as tf
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

Posteriormente, se importa el conjunto de datos con 50,000 palabras más frecuentes

In [11]:
print('Cargando los datos...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=50000)

X_combined = list(X_train) + list(X_test)
y_combined = list(y_train) + list(y_test)
df = pd.DataFrame({'Text': X_combined, 'Label': y_combined})

print("Datos cargados.")

Cargando los datos...
Datos cargados.


En este caso se optó por mezclar los datos de entrenamiento y prueba, para luego separarlos en 80% y 20% respectivamente. 

In [12]:
df.head()

,Text,Label
0,"[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...",1
1,"[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...",0
2,"[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...",0
3,"[1, 4, 18609, 16085, 33, 2804, 4, 2040, 432, 1...",1
4,"[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...",0


Más específicamente, como se puede observar a continuación, el dataset actual tiene los 50,000 datos. Por tal razón, **se logró la correcta importación del conjunto de datos con 50,000 palabras más frecuentes**.

In [13]:
print("Tamaño de los datos: ", df.shape)

Tamaño de los datos:  (50000, 2)


Con eso listo, se quiso convertir los datos a un formato más legible, por lo que se utilizó **word index** para convertir los datos a texto.

In [17]:
word_index = imdb.get_word_index()
reverse_word_index = {index: word for word, index in word_index.items()}

def decode_sequence(sequence):
    decoded_words = [reverse_word_index.get(index - 3, '') for index in sequence]
    decoded_words = [word for word in decoded_words if word != '']
    return ' '.join(decoded_words)


decoded = [decode_sequence(seq) for seq in df['Text']]
decoded_df = pd.DataFrame({'Text': decoded, 'Label': df['Label']})

decoded_df.head()

,Text,Label
0,this film was just brilliant casting location ...,1
1,big hair big boobs bad music and a giant safet...,0
2,this has to be one of the worst films of the 1...,0
3,the scots excel at storytelling the traditiona...,1
4,worst mistake of my life br br i picked this m...,0


#### `Pre-procesamiento`
- Secuencie y rellene las críticas para que todas tengan una longitud uniforme.
- De las críticas, extraiga características (features) adicionales, por ejemplo. la longitud de la crítica, la proporción de palabras positivas/negativas y cualquier otra que considere pueda ser útil.

Como primer paso, se limpió el dataset para eliminar stopwords (palabras vacías o irrelevantes).

In [23]:
# Eliminar palabras vacías (stop words)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
decoded_df["Text"] = decoded_df["Text"].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

# Realizar lematización o stemming (opcional)
stemmer = PorterStemmer()
decoded_df["Text"] = decoded_df["Text"].apply(lambda x: ' '.join(stemmer.stem(word) for word in x.split()))

# Mostrar el DataFrame
decoded_df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stefanoaragoni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Posteriormente, se procedió a tokenizar los datos, es decir, convertir las palabras a números. Esto se hizo con el fin de poder utilizar los datos en el modelo de LSTM.

Luego, se separaron los datos en el conjunto de entrenamiento y prueba. En este caso, se utilizó el 80% de los datos para entrenamiento y el 20% para prueba.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(decoded_df['Text'], decoded_df['Label'], test_size=0.2, random_state=42)

Posteriormente, se secuenció y rellenó los diferentes comentarios para que todos tengan una longitud uniforme. En este caso se utilizó una longitud 200. 

sequence.pad_sequences es una función de Keras que se utiliza para rellenar las secuencias. Si se pasa del largo máximo, se trunca la secuencia. Si es menor, se rellena con ceros.

In [9]:
X_train = pad_sequences(X_train, maxlen = 200)
X_test = pad_sequences(X_test, maxlen = 200)

#### `Modelo`
- Cree un modelo LSTM que acepte las características (features) adicionales junto con la secuencia de palabras.
- Intente usar una arquitectura más compleja, incorporando más capas LSTM, capas de Dropout para la regularización y tal vez alguna capa densamente conectada después de la LSTM. (ver también la referencia al final de este documento)

#### `Entrenamiento y Evaluación`
- Entrene su modelo con el conjunto de datos de entrenamiento y evalúe su desempeño con el conjunto de datos de prueba.

*Según Luis Furlán, el Jupyter Notebook cuenta como el informe en sí.*